In [2]:
import shutil
from pathlib import Path
from typing import List

import numpy as np
import pandas as pd
from tqdm import tqdm

import amazon_dataset

# Intialize Datasets

In [3]:
DATASET = 'Home_and_Kitchen'
VALIDATION_SIZE=0.15
RANDOM_SEED = 20230219
ALEXNET_IMAGE_FEATURES = Path(f'data/amazon/{DATASET}_alexnet_features.npz')
VIT_IMAGE_FEATURES = Path(f'data/amazon/{DATASET}_vit_features.npz')
CLIP_IMAGE_FEATURES = Path(f'data/amazon/{DATASET}_clipimage_features.npz')

CLIP_TEXT_FEATURES = Path(f'data/amazon/{DATASET}_cliptext_features.npz')
BERT_TEXT_FEATURES = Path(f'data/amazon/{DATASET}_bert_features.npz')
DEST_FOLDER = Path(f'data/{DATASET}')

assert ALEXNET_IMAGE_FEATURES.exists()
assert VIT_IMAGE_FEATURES.exists()
assert CLIP_IMAGE_FEATURES.exists()

assert BERT_TEXT_FEATURES.exists()
assert CLIP_TEXT_FEATURES.exists()


DEST_FOLDER.mkdir(exist_ok=True)

AssertionError: 

In [4]:
def generate_unique_ids(series: pd.Series) -> pd.Series:
    rng = np.random.default_rng(RANDOM_SEED)
    unique_ids = series.unique()
    return pd.Series(
        index=rng.permutation(unique_ids), 
        data=range(len(unique_ids))
    )

def df_stats(df: pd.DataFrame) -> str:
    n_items = len(df['asin'].unique())
    n_users = len(df['reviewerID'].unique())
    sparsity = 1. * len(df) / (n_users * n_items)
    return f'{n_items} items {n_users} users. Sparsity {sparsity * 100:.3f}%'

def save_numerized(
    reviews: pd.DataFrame, 
    uids: pd.Series, 
    product_ids: pd.Series,
    user_id_column: str,
    product_ids_column: str,
    columns: List[str],
    dest: Path
):
    """Save a Dataframe following userids and product_ids"""
    joined = reviews.join(uids.to_frame(user_id_column), on='reviewerID')
    assert joined[user_id_column].isna().sum() == 0
    
    joined = joined.join(product_ids.to_frame(product_ids_column), on='asin')
    assert joined[product_ids_column].isna().sum() == 0
    
    res = joined[columns]
    
    res.to_csv(dest, index=False)

    return res

def split_train_test_proportion(df: pd.DataFrame, test_prop=0.2):
    """
    Split the dataframe by reviewer and take exactly `test_prop` records
    for test and leave the rest for training
    """
    rng = np.random.default_rng(RANDOM_SEED)

    res = df.copy()
    res['rnd'] = rng.random(size=len(df))
    res['rnd_rank'] = res.groupby('reviewerID')['rnd'].rank(pct=True)
    condition = res['rnd_rank'] <= test_prop
    
    (_, train), (_, test) = res.groupby(condition)
    
    assert(len(set(train.index) & set(test.index)) == 0)

    return df.loc[train.index], df.loc[test.index]

In [5]:
reviews_df = amazon_dataset.reviews_df(DATASET)
reviews_df.sample(n=3)

,id,asin,reviewerID,reviewerName,overall,text,reviewTime,summary,verified,vote
139502,22958616,B000XBM1L2,A1IRP8WERRLVGR,louie,5.0,My grandson they fit fine. Could have been a ...,2018-05-23,My grandson they fit fine. Could have been a l...,True,NaN
42897,9047159,B00AFQZT4Q,A1WHGWRWCH8QAQ,Anonymous,5.0,"These are a little tiny bit large, but are so ...",2018-04-06,Nice shoes,True,NaN
73132,16379375,B00SU5244M,AJ2J83VRCIHYR,Amazon Customer,5.0,They're great. Dated a guy recently who had al...,2018-03-28,They're great. Dated a guy recently who had al...,True,NaN


# Generate Data for SEM-MacridVAE & DMRL

Some data stats

In [6]:
print(df_stats(reviews_df))

38493 items 23318 users. Sparsity 0.020%


Sort some users randomly

In [7]:
user_ids = generate_unique_ids(reviews_df['reviewerID'])
user_ids

A30NKRF3KBGA06        0
AL0XGCBE6Z22M         1
AMT5LF0TKY67C         2
A2BY8EVXA3NRHD        3
AWE6KR1ELIYQ3         4
                  ...  
A1MFBF49ZFMH2N    23313
A36AF5I7D0VO8F    23314
A35ZS7JT3G9B8     23315
A3GC94SEKQI3QU    23316
A185C12Y9XLYGY    23317
Length: 23318, dtype: int64

Same with products

In [8]:
item_ids = generate_unique_ids(reviews_df['asin'])
item_ids

B000B6AV7K        0
B0143D7EE4        1
B0105V2DEY        2
B014EY21H2        3
B005LUROIK        4
              ...  
B00A9R2P7A    38488
B017HK485S    38489
B008H7UKYY    38490
B006K6PJTK    38491
B01DUSBHZ0    38492
Length: 38493, dtype: int64

Split train test and validation by user

In [9]:
train_reviews, validation_test_reviews = split_train_test_proportion(reviews_df, test_prop=0.4)
validation_reviews, test_reviews = split_train_test_proportion(validation_test_reviews, test_prop=0.5)

In [10]:
len(train_reviews)

113836

len(validation_reviews)

In [11]:
len(test_reviews)

30728

Sanity checks. All users are in the train validation and test sets!

In [12]:
set(train_reviews['reviewerID']) == set(test_reviews['reviewerID']) ==  set(validation_reviews['reviewerID'])

True

Check that the train set and the validation set all have at least one item (same check as above)

In [13]:
pd.concat([
    train_reviews.groupby('reviewerID').size().to_frame('train_size'),
    validation_reviews.groupby('reviewerID').size().to_frame('val_size'),
    test_reviews.groupby('reviewerID').size().to_frame('test_size')
], axis=1).describe()

,train_size,val_size,test_size
count,23318.000000,23318.000000,23318.000000
mean,4.881894,1.474397,1.317780
std,3.529635,1.185903,1.091225
min,3.000000,1.000000,1.000000
25%,3.000000,1.000000,1.000000
50%,4.000000,1.000000,1.000000
75%,5.000000,2.000000,1.000000
max,61.000000,20.000000,20.000000


Generate users

In [14]:
user_ids.to_csv(DEST_FOLDER / 'users.txt')
item_ids.to_csv(DEST_FOLDER / 'items.txt', header=['item_id'])

In [15]:
NUMERIZED_COMMON_PARAMS = {
    'uids': user_ids, 
    'product_ids': item_ids,
    'user_id_column': 'user',
    'product_ids_column': 'item',
    'columns': ['user', 'item']
}

save_numerized(
    train_reviews, 
    dest=DEST_FOLDER / 'train.txt',
    **NUMERIZED_COMMON_PARAMS
)

,user,item
0,14891,30409
4,19073,29339
6,21394,38246
10,22275,38246
14,6565,38246
...,...,...
178934,5195,6878
178935,418,7919
178937,11611,5762
178938,16539,5762


In [16]:
save_numerized(
    validation_reviews, 
    dest=DEST_FOLDER / 'validation.txt',
    **NUMERIZED_COMMON_PARAMS
)
'Saved validation data'

'Saved validation data'

In [17]:
save_numerized(
    test_reviews, 
    dest=DEST_FOLDER / 'test.txt',
    **NUMERIZED_COMMON_PARAMS
)
'Saved test data'

'Saved test data'

In [18]:
def copy_features(
        item_ids: pd.Series, 
        features_file: Path
):
    print('Opening file')
    with np.load(features_file) as features:
        some_embedding = next(iter(features.values()))
        embedding_shape, = some_embedding.shape
        array_shape = (len(item_ids),  embedding_shape)
        print(f'Initializing array {array_shape}')
        res = np.full(array_shape,  fill_value=np.nan)

        for asin, idx in tqdm(item_ids.items(), 
                total=len(item_ids), unit_scale=True, unit='items', 
                desc='Copying features'):
            value = features[asin]
            
            assert np.isnan(value).sum() == 0, "Feature has NaN Values"
            
            res[idx, :] = value

        return res


image_features_array = copy_features(
    item_ids=item_ids, 
    features_file=CLIP_IMAGE_FEATURES
)
image_features_array.shape

Opening file
Initializing array (38493, 768)


Copying features:  24%|██▍       | 9.42k/38.5k [00:18<00:58, 499items/s]


KeyboardInterrupt: 

In [ ]:
IMAGE_EMBED_DEST = DEST_FOLDER / 'embed_image.npy'
np.save(IMAGE_EMBED_DEST, image_features_array)

print(f'{str(IMAGE_EMBED_DEST)}: {IMAGE_EMBED_DEST.stat().st_size // 2**20}MiB')

data/Clothing_Shoes_and_Jewelry/embed_image.npy: 225MiB


In [ ]:
text_features_array = copy_features(
    item_ids=item_ids, 
    features_file=CLIP_TEXT_FEATURES
)
text_features_array.shape

Opening file
Initializing array (38493, 768)


Copying features: 100%|██████████| 38.5k/38.5k [01:02<00:00, 615items/s]


(38493, 768)

In [ ]:
TEXT_EMBED_DEST = DEST_FOLDER / 'embed_text.npy'
np.save(TEXT_EMBED_DEST, text_features_array)

print(f'{str(TEXT_EMBED_DEST)}: {TEXT_EMBED_DEST.stat().st_size // 2**20}MiB')

data/Clothing_Shoes_and_Jewelry/embed_text.npy: 225MiB


# Generate images

In [68]:
products_df = amazon_dataset.products_df(DATASET)
products_images_dir = amazon_dataset.product_images_dir(DATASET)
products_images_dir

PosixPath('data/amazon/Clothing_Shoes_and_Jewelry_product_images')

In [122]:
import cv2

def convert_image(src: str, dst: str):
    min_white = np.array([235, 235, 235], np.uint8)
    white = np.array([255, 255, 255], np.uint8)
    img = cv2.imread(src)
    
    mask = 255 - cv2.inRange(img, min_white, white)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
    res = cv2.bitwise_and(img, img, mask=mask)
    cv2.imwrite(dst, res)

def save_images(products_df: pd.DataFrame, item_ids: pd.Series, dest: Path):
    products_by_asin = products_df.set_index('asin')
    joined  = item_ids.to_frame('item_id').join(
        products_by_asin, validate='one_to_one'
    )
    dest.mkdir(exist_ok=True)
    
    no_images = 0
    
    progress = tqdm(
        joined.itertuples(), 
        desc='Copying images', 
        total=len(joined), 
        unit_scale=True, unit='items'
    )
    for row in progress:
        if not isinstance(row.image_slug, list):
            print('Not a list')
            break
        if len(row.image_slug) == 0:
            no_images += 1
            progress.set_postfix_str(f'No images: {no_images}', refresh=False)
            continue
        slug = row.image_slug[0]
        src = products_images_dir / f"{slug}.jpg"
        assert src.exists()
        dst = dest / f'{row.item_id}.png'
        convert_image(str(src), str(dst))

#save_images(products_df, item_ids,  DEST_FOLDER / 'images')

Copying images: 100%|██████████| 38.5k/38.5k [04:44<00:00, 135items/s, No images: 234]


# Generate categories

In [ ]:
# categories_df = amazon_dataset.product_categories_df(DATASET)
# products_df = amazon_dataset.products_df(DATASET)

In [ ]:
# categories_id = generate_unique_ids(categories_df['name'])
# categories_id.to_csv(DEST_FOLDER / 'categories.txt', header=['category_id'])
# categories_id

Calf Circumference up to 17 inches                                                                                                                                                                      0
pull-on style makes getting dressed hassle-free                                                                                                                                                         1
GELForm removable insole                                                                                                                                                                                2
Material: 95% Polyester, 5% Spandex,high quality material, soft and comfortable.                                                                                                                        3
size:US size,please choose the size according size chart,if you couldn't ensure the size,please email us your measurements:height,bust,waist and hip,then we will check which size fit you well.

In [ ]:
def build_categories_matrix(
        categories_df: pd.DataFrame, 
        products_df: pd.DataFrame, 
        item_ids: pd.Series, 
        categories_id: pd.Series
    ):
    MIN_SUPPORT = 0.01
    MAX_SUPPORT = 0.9
    
    # Categories only with not too many or too few items
    categories_support = categories_df['name'].value_counts() 

    allowed_categories_names = categories_support[1:8].index

    print("How many allowed categories")
    print(len(allowed_categories_names))
    print(allowed_categories_names)
    
    filtered_categories = categories_df.loc[
        categories_df['name'].isin(allowed_categories_names)
    ]

    products_df = products_df.join(
        item_ids.to_frame('item_id'), on='asin', validate='1:1')

    res = filtered_categories.set_index('product_id').join(
        products_df, validate='m:1')
    res = res.reset_index(drop=True)
    
    # Check all categories are in categories_id
    res = res.join(categories_id.to_frame('category_id'), on='name')
    assert res['category_id'].isna().sum() == 0

    items_with_no_category = set(item_ids) - set(res['item_id'].unique())
    print(f"items with no category: {len(items_with_no_category)}")
    
    return products_df.loc[
        products_df['asin'].isin(item_ids[items_with_no_category].index)
    ]

    return res[['item_id', 'category_id']]

# categories_matrix = build_categories_matrix(
#     categories_df, products_df, item_ids, categories_id
# )
#categories_matrix
#categories_matrix.to_csv(DEST_FOLDER / 'categorical.txt', index=False)